In [26]:
import os.path as osp
import os

#torch 전에 할댕해줘야함
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]= "1"

In [27]:


import torch
from tqdm import tqdm

from torch_geometric.datasets import AmazonBook, MovieLens100K
from torch_geometric.nn import LightGCN
from torch_geometric.utils import degree
from torch_geometric.data import InMemoryDataset

In [28]:


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [29]:
# ml 1m 
path = osp.join('./', 'data', 'ML1M')
dataset = MovieLens100K(path)
dataset.process()
data = dataset[0]
num_users, num_books = data['user'].num_nodes, data['movie'].num_nodes
data = data.to_homogeneous().to(device)
# 이새끼 호모지니어스로 바꾸면 엣지 라벨 순서가 바뀌어 버림
data.edge_label_index = torch.stack([data.edge_label_index[1],data.edge_label_index[0]], dim = 0)
data.edge_label_index

tensor([[   5,    9,   11,  ...,  933,    9,  681],
        [1682, 1682, 1682,  ..., 2140, 2141, 2143]], device='cuda:0')

In [30]:
# 아마존
path = osp.join('./', 'data', 'Amazon')
dataset = AmazonBook(path)
data = dataset[0]
num_users, num_books = data['user'].num_nodes, data['book'].num_nodes
data = data.to_homogeneous().to(device)


In [31]:
import numpy as np
import pandas as pd
from typing import Callable, List, Optional

import os.path as osp
import os

import torch

from torch_geometric.data import HeteroData, InMemoryDataset, download_url
class Gowalla(InMemoryDataset):
    url = 'https://snap.stanford.edu/data/loc-gowalla_totalCheckins.txt.gz'
  

    def __init__(
        self,
        root: str,
        transform: Optional[Callable] = None,
        pre_transform: Optional[Callable] = None,
        force_reload: bool = False,
        core: int = None,
    ) -> None:
        super().__init__(root, transform, pre_transform,
                         force_reload=force_reload)
        self.core = core
        self.load(self.processed_paths[0], data_cls=HeteroData)
    @property
    def raw_file_names(self) -> List[str]:
        return ['loc-gowalla_totalCheckins.txt', 'filtered_total.txt', 'user_id_map.txt', 'item_id_map.txt', 'train.txt', 'test.txt']

    @property
    def processed_file_names(self) -> str:
        return 'data.pt'
    
    @property
    def ziped_file_name(self) -> str:
        return 'loc-gowalla_totalCheckins.txt.gz'

    def download(self) -> None:
        import gzip

        if not osp.isfile(osp.join(self.root, 'loc-gowalla_totalCheckins.txt.gz')):
            download_url(f'{self.url}', self.root)
        if not osp.isfile(osp.join(self.root,'raw/loc-gowalla_totalCheckins.txt')): 
            os.system(f'gzip -d {osp.join(self.root,"loc-gowalla_totalCheckins.txt.gz")}')
            os.system(f'mv {osp.join(self.root,"raw/loc-gowalla_totalCheckins.txt")} {osp.join(self.root,"loc-gowalla_totalCheckins.txt.gz")}')
    
    def core_filter(self, df, threshold):

        df = pd.read_csv(osp.join(self.raw_dir, self.raw_file_names[0]),sep = '\t', names = ['user', 'time', 'long', 'lat', 'item'])
        
        filtered_df = self.filtering(df, threshold)
        processed_df, user_id, item_id  = self.refactoring_from_0(filtered_df)

        processed_df.to_csv(osp.join(self.raw_dir, self.raw_file_names[1]),sep=" ", index=False, header=None)
        user_id.to_csv(osp.join(self.raw_dir, self.raw_file_names[2]),sep=" ", index=False, header=None)
        item_id.to_csv(osp.join(self.raw_dir, self.raw_file_names[3]),sep=" ", index=False, header=None)

        return processed_df

    def refactoring_from_0(self, df):
        out_df = pd.DataFrame() 
        
        original_uid = np.sort(df['user'].unique())
        original_iid = np.sort(df['item'].unique())

        u_range = range(len(original_uid))
        i_range = range(len(original_iid))

        uid_mapping = { o_id: n_id for o_id, n_id in zip(original_uid, u_range)} # 원래 유저 아이디 (중간중간 비어있음) : 순서대로 유저 아이디
        iid_mapping = { o_id: n_id for o_id, n_id in zip(original_iid,i_range)} # 원래 아이템 아이디 : 순서대로 아이템 아이디

        uid_map = pd.DataFrame({'o_id' : list(uid_mapping.keys()), 'n_id' : list(uid_mapping.values())})

        iid_map = pd.DataFrame({'o_id' : list(iid_mapping.keys()),'n_id':list(iid_mapping.values())})


        out_df['user'] = df['user'].map(uid_mapping)
        out_df['item'] = df['item'].map(iid_mapping)
        out_df['time'] = df['time']
        return out_df, uid_map, iid_map

    def filtering(self, df, threshold) :
        fdf = df.drop_duplicates(subset=['user', 'item'], keep='first')
        while fdf.user.value_counts().min() < threshold or fdf.item.value_counts().min() < threshold:
            df_item = fdf.groupby('item').count()
            df_item = df_item[df_item.user < threshold]
            li = df_item.index.to_list()
            fdf = fdf.drop(fdf.loc[fdf.item.isin(li)].index)

            df_usr = fdf.groupby('user').count()
            df_usr = df_usr[df_usr.item < threshold]
            li = df_usr.index.to_list()
            fdf = fdf.drop(fdf.loc[fdf.user.isin(li)].index)

            # print(f"Total Edges : {len(fdf)}\nTotal User : {len(fdf['user'].unique())}\nTotal item : {len(fdf['item'].unique())} \
            #             \nMin Interaction Per user : {fdf.user.value_counts().min()} \
            #             \nMax Interaction Per user : {fdf.user.value_counts().max()} \
            #             \nAvg Interaction Per user : {fdf.user.value_counts().mean()}\
            #             \nMin Interaction Per item : {fdf.item.value_counts().min()} \
            #             \nMax Interaction Per item : {fdf.item.value_counts().max()} \
            #             \nAvg Interaction Per item : {fdf.item.value_counts().mean()}")
        
        fdf = fdf.reset_index().drop(columns = ['index'])
        return fdf

    def process(self) -> None:
        from sklearn.model_selection import train_test_split

        data = HeteroData()
        attr_names = ['edge_index', 'edge_label_index']
        # Process number of nodes for each node type:
        node_types = ['user', 'item']

        if osp.isfile(osp.join(self.root,f"raw/{self.raw_file_names[4]}")) \
            and osp.isfile(osp.join(self.root,f"raw/{self.raw_file_names[5]}")):
            # Process edge information for training and testing:

            for path, node_type in zip(self.raw_paths[2:4], node_types):
                df = pd.read_csv(path, sep=' ', header= None)
                data[node_type].num_nodes = len(df)

            for path, attr_name in zip(self.raw_paths[4:], attr_names):
                temp_df = pd.read_csv(path, names = ['user', 'item', 'time'], header = None)
                rows = temp_df['user'].values
                cols = temp_df['item'].values
                index = torch.tensor([rows, cols])

                data['user', 'rates', 'item'][attr_name] = index
                if attr_name == 'edge_index':
                    data['item', 'rated_by', 'user'][attr_name] = index.flip([0])

        else:
            if osp.isfile(osp.join(self.root,f"raw/{self.raw_file_names[1]}")) \
                and osp.isfile(osp.join(self.root,f"raw/{self.raw_file_names[2]}")) \
                    and osp.isfile(osp.join(self.root,f"raw/{self.raw_file_names[3]}")):
                df = pd.read_csv(osp.join(self.raw_dir, self.raw_file_names[1]), sep = " ", names = ['user', 'item', 'time'], header = None)
            else:
                df = pd.read_csv(osp.join(self.raw_dir, self.raw_file_names[0]),  names = ['user', 'time', 'long', 'lat', 'item'], header = None)
                self.core_filter(df, self.core)
                df = pd.read_csv(osp.join(self.raw_dir, self.raw_file_names[1]), sep = " ", names = ['user', 'item', 'time'], header = None)

            tr, test = train_test_split(df, test_size = 0.2)
            tr.to_csv(osp.join(self.raw_dir, self.raw_file_names[4]), index = False,header = False)
            test.to_csv(osp.join(self.raw_dir, self.raw_file_names[5]), index = False, header = False)

            for path, node_type in zip(self.raw_paths[2:4], node_types):
                df = pd.read_csv(path, sep=' ', header= None)
                data[node_type].num_nodes = len(df)

            for temp_df, attr_name in zip([tr,test],attr_names):
                rows = temp_df['user'].values
                cols = temp_df['item'].values
                index = torch.tensor([rows, cols])

                data['user', 'rates', 'item'][attr_name] = index
                if attr_name == 'edge_index':
                    data['item', 'rated_by', 'user'][attr_name] = index.flip([0])

        if self.pre_transform is not None:
            data = self.pre_transform(data)

        self.save([data], self.processed_paths[0])


In [32]:
# Gowalla
path = osp.join('./', 'data', 'Gowalla')
dataset = Gowalla(path)
dataset.process()
data = dataset[0]
num_users, num_books = data['user'].num_nodes, data['item'].num_nodes
data = data.to_homogeneous().to(device)

In [33]:
dataset[0]

HeteroData(
  user={ num_nodes=29858 },
  item={ num_nodes=40988 },
  (user, rates, item)={
    edge_index=[2, 821971],
    edge_label_index=[2, 205493],
  },
  (item, rated_by, user)={ edge_index=[2, 821971] }
)

In [34]:
# Use all message passing edges as training labels:
batch_size = 8192
mask = data.edge_index[0] < data.edge_index[1]
train_edge_label_index = data.edge_index[:, mask]
train_loader = torch.utils.data.DataLoader(
    range(train_edge_label_index.size(1)),
    shuffle=True,
    batch_size=batch_size,
)

model = LightGCN(
    num_nodes=data.num_nodes,
    embedding_dim=64,
    num_layers=2
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [35]:
def train():
    total_loss = total_examples = 0

    for index in train_loader:
        # Sample positive and negative labels.
        pos_edge_label_index = train_edge_label_index[:, index]
        neg_edge_label_index = torch.stack([
            pos_edge_label_index[0],
            torch.randint(num_users, num_users + num_books,
                          (index.numel(), ), device=device)
        ], dim=0)
        edge_label_index = torch.cat([
            pos_edge_label_index,
            neg_edge_label_index,
        ], dim=1)

        optimizer.zero_grad()
        pos_rank, neg_rank = model(data.edge_index, edge_label_index).chunk(2)

        loss = model.recommendation_loss(
            pos_rank,
            neg_rank,
            node_id=edge_label_index.unique(),
            lambda_reg = 0.001
        )
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * pos_rank.numel()
        total_examples += pos_rank.numel()

    return total_loss / total_examples

In [36]:
@torch.no_grad()
def test(k: int):
    emb = model.get_embedding(data.edge_index)
    user_emb, book_emb = emb[:num_users], emb[num_users:]

    precision = recall = total_examples = 0
    for start in range(0, num_users, batch_size):
        end = min(start + batch_size, num_users)
        logits = user_emb[start:end] @ book_emb.t()
        # Exclude training edges:
        mask = ((train_edge_label_index[0] >= start) &
                (train_edge_label_index[0] < end))
        logits[train_edge_label_index[0, mask] - start,
                train_edge_label_index[1, mask] - num_users] = float('-inf')

        # Computing precision and recall:
        ground_truth = torch.zeros_like(logits, dtype=torch.bool)
        mask = ((data.edge_label_index[0] >= start) &
                (data.edge_label_index[0] < end))
        ground_truth[data.edge_label_index[0, mask] - start,
                        data.edge_label_index[1, mask] - num_users] = True
        node_count = degree(data.edge_label_index[0, mask] - start,
                                num_nodes=logits.size(0))

        topk_index = logits.topk(k, dim=-1).indices
        isin_mat = ground_truth.gather(1, topk_index)

        precision += float((isin_mat.sum(dim=-1) / k).sum())
        recall += float((isin_mat.sum(dim=-1) / node_count.clamp(1e-6)).sum())
        total_examples += int((node_count > 0).sum())

    return precision / total_examples, recall / total_examples

In [25]:
best = 0
best_model = model.get_embedding(data.edge_index).detach()
best_init = model.embedding.weight.detach()
p = 0
for epoch in tqdm(range(0, 1000)):
    loss = train()
    precision, recall = test(k=20)
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Precision@20: '
            f'{precision:.4f}, Recall@20: {recall:.4f}')
        if recall > best:
            p = 0
            best_model = model.get_embedding(data.edge_index).detach()
            best_init = model.embedding.weight.detach()
            best = recall
        else:
            p += 1
            if p > 10:
                break

  0%|          | 1/1000 [00:01<31:59,  1.92s/it]

Epoch: 000, Loss: 0.6382, Precision@20: 0.0194, Recall@20: 0.0641


  1%|          | 11/1000 [00:21<31:28,  1.91s/it]

Epoch: 010, Loss: 0.1418, Precision@20: 0.0273, Recall@20: 0.0938


  2%|▏         | 21/1000 [00:40<31:12,  1.91s/it]

...

 54%|█████▍    | 541/1000 [17:13<14:37,  1.91s/it]

Epoch: 540, Loss: 0.0578, Precision@20: 0.0496, Recall@20: 0.1699


 55%|█████▌    | 551/1000 [17:32<14:15,  1.91s/it]

Epoch: 550, Loss: 0.0577, Precision@20: 0.0496, Recall@20: 0.1695


 56%|█████▌    | 557/1000 [17:44<14:00,  1.90s/it]